In [ ]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [ ]:
# Procesa la variable para todos los años.
def proc_mean(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[0] + var_2 + "/mexico_" 
        + var_2 + "." + str(year) + ".nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df[ df["Año"] == year ].copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

# Procesa la variable para todos los años.
def proc_sum(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[0] + var_2 + "/mexico_" 
        + var_2 + "." + str(year) + ".nc" ) as ds:
        df_y = df[ df["Año"] == year ].copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().sum()[0] )
        return df_y

In [ ]:
# Procesa la variable para todos los años.
def proc_mean_f():
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k + 1] + var_2 + "/mexico_" 
        + var_2 + "." + str(f) + ".nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df.copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

# Procesa la variable para todos los años.
def proc_sum_f():
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k + 1] + var_2 + "/mexico_" 
        + var_2 + "." + str(f) + ".nc" ) as ds:
        df_y = df.copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ df["Municipio"] == i,
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().sum()[0] )
        return df_y

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 

RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
years = list(range(2010, 2017))

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path_drive + "data/" + x + "/" for x in dirs]
folder = "vars/"
path_d = [path_drive + x + "/" for x in dirs]

# Variables de CHIRTS y CHIRPS.
vars = ["Tmax", "Tmin", "Tmean", "HDD_mean", 
        "CDD_mean", "HDD_p10", "CDD_p90",
        "Pre", "Pre_Tmean", "Poblacion", "PIB"]

# Nombre de los archivos
f_mask = "municipios_mask.nc"
f_data = "data_0.csv"

# Máscara para los municipios.
mask = xr.open_dataset( path_drive + f_mask )

In [ ]:
# Escenario a procesar, 0: presente, 1: futuro.
e = 1

# Variable a procesar, con e = 0, 0 - 8; con e = 1, 0 - 10.
n = 0
var_2 = vars[n]

if var_2 in vars[3:7]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var_2)

# Años a procesar.
year = list(range(2010, 2017))

# Procesamos los años en paralelo.
pool = Pool()
if e == 0:
    # Abrimos los archivos a utilizar.
    df = pd.read_csv( path_data[0] + f_data )
    var_1 =  df.columns[4:-9][n]

    if var_2 in vars[3:9]:
        df_r = pool.map(proc_sum, year)
    else:
        df_r = pool.map(proc_mean, year)

    # Concatenamos los resultados de cada año.
    for p in df_r:
        df.loc[df["Año"] == p["Año"].max() ] = p

    # Guardamos el resultado como un nuevo archivo.
    df.to_csv(path_data + "data_" + str(n + 1)
        + ".csv", index = False)
else: 
    for d in range(len(path_d[1:])):
        # Abrimos los archivos a utilizar.
        df = pd.read_csv( path_data[k + 1] + f_data )
        var_1 =  df.columns[4:-9][n]

        for f in fut:
            if var_2 in vars[3:9]:
                df_r = pool.map(proc_sum)
            else:
                df_r = pool.map(proc_mean)

            # Concatenamos los resultados de cada año.
            for p in df_r:
                df.loc[df["Año"] == p["Año"].max() ] = p

            # Guardamos el resultado como un nuevo archivo.
            df.to_csv(path_data[k + 1] + "data_" + str(n + 1)
                  + "." + str(f) + ".csv", index = False)

ValueError: ignored